In [1]:
import os
import re
import boto3
import itertools
import json

import pandas as pd
import numpy as np
from sagemaker.session import Session

In [2]:
# create a darker background template (better for my eyes)
from jupyterthemes.stylefx import set_nb_theme
# set_nb_theme('chesterish')

## Helper Functions & Classes

In [95]:
# small class for holding "special" print flags
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [204]:
def QAcheck(tests, checks, func):
    """
    Quality-assurance checks for expected outputs, and function outputs
    """
    for index, test in enumerate(tests):
        value = func(test)
        
        if type(value) == np.ndarray:  
            if (value == checks[index]).all():
                print('Correct Match\t{} = {}'.format(tests[index], checks[index]))
        else:
            if (value is checks[index]):
                print('Correct Match\t{} = {}'.format(tests[index], checks[index]))
            elif (value == checks[index]):
                print('Correct Match\t{} = {}'.format(tests[index], checks[index]))
            else:
                print(color.BOLD,color.RED,'ERROR MATCH\t{} /= {}'.format(tests[index], checks[index]),color.END)

In [165]:
def cleanNumeric(value) -> float:
    """
    This function converts a string to a numeric quantity, handles weird string format
    :param: value, string value with hidden numeric quanity  
    :return: floating point values
    
    Complexity -> O(n)
    
    e.g.
        In[0]: $ 19,225     ->   Out[0]: 19255
        In[0]: $ 19,225.76  ->   Out[0]: 19255.76
        
    """
    
    assert type(value) is str or int or np.ndarray, 'Value must be of type string, integer, float or numpy array'
    
    # checks to see what type of value is being provided
    operator = type(value)
    
    def num_strip(number):
        """
        Nested function for extracting numerical quantities
        """
        try:
            # some accounting formats take () to be negative numbers
            if number[0] == '(':
                number = '-' + number

            # perform regex operation scanning for only numeric quantities/identifiers
            cleanValue = re.sub("[^0-9|.|-]", "", number)

            # last check against poor lagging formats e.g. "." or "-" to return nan or floating-point number
            try: 
                return float(cleanValue)
            except ValueError: 
                return np.nan
            
        except (TypeError, IndexError):
            return np.nan
    
    # if provided a string, perform regex operation 
    if (operator is str) and (len(value) > 0):
        return num_strip(value)
    
    # if operator is integer then simply return the value, no need to modify 
    elif (operator is int):
        return value
    
    # if operator is numpy array then we perform a extraction per element in array
    elif (operator is np.ndarray):
        vFunc = np.vectorize(num_strip)      # vectorize function to apply to numpy array
        cleanValue = vFunc(value)            # apply vector function
        return cleanValue
    
    else:
        return np.nan

In [205]:
tests = np.array(['', '$.', '$ -', '.3', '128,123', '-$24,613', '(21,843)', '$212,223',
                 np.array(['$13,300', '-$2344', '$23,421']), np.array(['$13,300', np.nan, '$23,421']),
                 np.array([np.nan, np.nan])])
expectation = np.array([np.nan, np.nan, np.nan, 0.3, 128123.0, -24613.0, -21843.0, 212223.0, 
                        [13300., -2344., 23421.], [13300., np.float64(np.nan), 23421.], 
                        [np.float64(np.nan), np.float64(np.nan)]])

QAcheck(tests, expectation, cleanNumeric)

Correct Match	 = nan
Correct Match	$. = nan
Correct Match	$ - = nan
Correct Match	.3 = 0.3
Correct Match	128,123 = 128123.0
Correct Match	-$24,613 = -24613.0
Correct Match	(21,843) = -21843.0
Correct Match	$212,223 = 212223.0
Correct Match	['$13,300' '-$2344' '$23,421'] = [13300.0, -2344.0, 23421.0]


In [5]:
def regexCheck(string:str, searchTerm:str):
    """
    Checks a regex expression for a given string construct
    string (type str)
        A string to perform a regex search on e.g. "Cash and cash equiavalents"
    searchTerm (type str)
        A regex expression to execute a search e.g. "^Cash"
        
    return: the searched term in question
    """
    try:
        s = re.search(searchTerm, string, flags=re.I)
        return s.string
    except AttributeError:
        return None

In [206]:
# sample = "Total Liabilities stockholder's equity"
sample = "MEMBER'S EQUITY"
regexCheck(sample, '^(?!.*liabilities).*equity$')

"MEMBER'S EQUITY"

In [8]:
def tMatch(totalA:float, totalLE:float):
    # helps match the accounting equation assets = libabilities + equities 
    try:
        multiplier = totalA / totalLE
        
        # if multiplier is zero, then total asset column is zero, we return nothing (imperfect)
        if multiplier == 0: return (None, 'Asset table is missing')
        
        # if multiplier is non-negative we accept only 1 or multiples of 10 (perfect and semi-perfect)
        elif (multiplier == 1) or (multiplier%10 == 0): return (multiplier, 'Perfect/Semi-Perfect match')
        
        # all other multipliers, should return nothing (number mismatch) 
        else: return (None, 'Accounting Error')
        
    # if total liabilites doesn't exist, but total assets either exists or does not exist  
    except ZeroDivisionError:
        return (None, "Either Liabilities/Equities doesn't exist or both Total Assets and Liabilities/Equities missing")

In [9]:
print(tMatch(totalA=0, totalLE=100))
print(tMatch(totalA=0, totalLE=0))
print(tMatch(totalA=100, totalLE=0))
print(tMatch(totalA=100, totalLE=100))
print(tMatch(totalA=10000, totalLE=100))
print(tMatch(totalA=102020, totalLE=11))

(None, 'Asset table is missing')
(None, "Either Liabilities/Equities doesn't exist or both Total Assets and Liabilities/Equities missing")
(None, "Either Liabilities/Equities doesn't exist or both Total Assets and Liabilities/Equities missing")
(1.0, 'Perfect/Semi-Perfect match')
(100.0, 'Perfect/Semi-Perfect match')
(None, 'Accounting Error')


## Interact with System Environment

In [207]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="Output/BalanceSheet/"

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

In [208]:
print('Total cleaned .csv files total {}'.format(paths.size-1))

Total cleaned .csv files total 12918


## Test Case Example

In [12]:
selections = np.random.choice(paths[1:], 100)

In [13]:
index = 40

In [14]:
# retrieving downloaded files from s3 bucket
s3.download_file(bucket, selections[index], 'temp.pdf')

df = pd.read_csv('temp.pdf')

In [15]:
# complete .csv file read from the 
df

,Unnamed: 0,0,1
0,0,Cash,"$ 39,499"
1,1,Due from Clearing Broker,"242,638,316"
2,2,"Securities owned, at market value (Note 5)","38,087,111"
3,3,JBO investment,"50,000"
4,4,Equipment (net of accumulated depreciation of ...,"151,499"
5,5,Leasehold improvements (net of accumulated amo...,"31,172"
6,6,TOTAL ASSETS,"$ 280,997,597"
7,7,LIABILITIES AND MEMBERS' EQUITY,NaN
8,8,Accounts payable and accrued expenses,"$ 128,392"
9,9,"Securities sold, not yet purchased, at market","243,570,454"


In [16]:
# clean dataframes for only the items and their immediate values
cleanDf = df[df.columns[1:3]].dropna() 

In [17]:
cleanDf['1'] = cleanDf['1'].apply(cleanNumeric)

In [18]:
cleanDf

,0,1
0,Cash,39499.0
1,Due from Clearing Broker,242638316.0
2,"Securities owned, at market value (Note 5)",38087111.0
3,JBO investment,50000.0
4,Equipment (net of accumulated depreciation of ...,151499.0
5,Leasehold improvements (net of accumulated amo...,31172.0
6,TOTAL ASSETS,280997597.0
8,Accounts payable and accrued expenses,128392.0
9,"Securities sold, not yet purchased, at market",243570454.0
10,TOTAL LIABILITIES,243698846.0


In [19]:
print('Data for {}'.format(selections[index]))

Data for Output/BalanceSheet/1049854-03.csv


In [20]:
def extraction(df:pd.DataFrame, file:str) -> pd.DataFrame:
    """
    Extract specific line items from cleaned dataframes (balance sheets)
    :param: df (type pd.DataFrame)
        The dataframe from
    :param: file (type str)
        The file string stored in the s3 bucket
    
    :return: A dataframe subset with critical lines extracted
    """
    
    # regex expression for searching for line items 
    keySearch = ['^cash', 'deposits', 'receivable', '^total assets', 'payable', 'total liabilities$', 'common stock', 
                 'preferred stock', 'earnings', '^(?!.*liabilities).*equity$|^(?!.*liabilities).*equity:$', 
                 '^total liabilities.*equity$|^total liabilities.*equity:$']
    
    # key names reflecting the corresponding the regex names
    keyNames = ['Cash & Equivalents', 'Deposits', 'Receivables', 'Total Assets', 'Payables', 'Total Liabilities',
                'Common Stock', 'Preferred Stock', 'Earnings', 'Total Equity', 'Total Liabilities & Equity']
    
    # concat the list of dataframe for each extraction 
    series = {}
    
    # creating two rows to track the CIK and year information released
    cik, year = file.split('/')[-1].split('-')
    
    series['CIK'] = cik          # CIK number for firm 
    series['Year'] = year[:2]    # Year for firm filing  
    
    # select key for search names
    for i, key in enumerate(keySearch):
        # filter regex for corresponding string expression
        filterSet = df[df.columns[0]].str.contains(key, regex=True, flags=re.IGNORECASE)
        
        # matching names for search and constructing a row (dictionary form)
        # filter dataframe and sum corresponding data column
        series[keyNames[i]] = df[filterSet][df.columns[1]].sum()
  
    # match the T-table, asset/liabilities/equity for rows
    mul, msg = tMatch(series['Total Assets'], series['Total Liabilities & Equity'])
    
    if mul != None:
        series['Total Liabilities & Equity'] = series['Total Liabilities & Equity'] * mul
        
        # computing the series for other available line item
        series['Other Assets'] = series['Total Assets'] - series['Cash & Equivalents'] - series['Deposits'] - series['Receivables']
        series['Other Liabilities'] = series['Total Liabilities'] - series['Payables']
        series['Other Equity'] = series['Total Equity'] - series['Common Stock'] - series['Preferred Stock'] - series['Earnings']

        # convert the filtered dictionaries to a dataframe
        comboDF = pd.DataFrame.from_dict(series, orient='index')

        return comboDF.transpose()
    else:
        return msg

In [21]:
extraction(cleanDf, selections[index])

,CIK,Year,Cash & Equivalents,Deposits,Receivables,Total Assets,Payables,Total Liabilities,Common Stock,Preferred Stock,Earnings,Total Equity,Total Liabilities & Equity,Other Assets,Other Liabilities,Other Equity
0,1049854,03,39499,0,0,2.80998e+08,128392,2.43699e+08,0,0,0,3.72988e+07,2.80998e+08,2.80958e+08,2.4357e+08,3.72988e+07


# X-17A-5 Database Construction

In [76]:
# subsample of path-objects to observe
obsRange = paths[1:500]

## Balance Sheet item extraction
**We start by retrieving all asset line items as well as liability & equity line items**

In [209]:
def balanceSheetLines(fileNames:np.ndarray) -> tuple:
    """
    Retrieving balance sheet information line item names for s3 files
    :param: (type np.ndarray)
        An array of file names for .csv files from s3
        
    :return: (type tuple)
        Return a tuple of arrays; left is the asset items, and right is the liability & equity items 
    """
    
    # initialize asset and liability list to store items
    Asset = []
    LiabilityandEquity = []

    # iterate through files from s3 bucket 
    for file in fileNames:
        
        # download temporary file from s3 bucket
        s3.download_file(bucket, file, 'temp.pdf')
        df = pd.read_csv('temp.pdf')
        
        n = df.columns.size   # the number of columns in read dataframe
        
        if n > 1: # if there is more than 1 column we continue examination 
            
            arr = df[df.columns[1]].dropna().values     # all line item for balance sheet (second column)
            
            # iterate through the line items as provided by the array
            for i, item in enumerate(arr):
                
                # search string for presence of word 'liabilties' or 'liability' 
                val = re.search('liabilities|liability ', item, flags=re.I)
                
                # if we find the term 'liability' or 'liabilities' we split the line items and break inner loop
                if val is not None:
                    
                    # partition by the enumerated index for asset and liability portions
                    Asset.append(arr[:i])
                    LiabilityandEquity.append(arr[i:])
                    break
                    
        # remove local file after it has been created
        os.remove('temp.pdf')
    
    # flatten series for all values
    left = np.hstack(np.array(Asset))
    right = np.hstack(np.array(LiabilityandEquity))
    
    return (left, right)

In [78]:
leftSide, rightSide = balanceSheetLines(obsRange)

In [211]:
# storing unique list of asset items and liability line items
with open('assetLines.txt', 'w') as f: json.dump(list(set(leftSide)), f)
with open('liabilityLines.txt', 'w') as f: json.dump(list(set(rightSide)), f)

print('All line items have been stored from balance sheet reading\n\tAsset items are stored\n\tLiability & Equity items are stored')

All line items have been stored from balance sheet reading
	Asset items are stored
	Liability & Equity items are stored


## Unstructured Data formulation
**Use the extracted asset and liaiblity lines to construct a large panel dataset, where line items are columns**

In [80]:
with open('assetLines.txt', 'r') as f: assetSide = json.loads(f.read())
with open('liabilityLines.txt', 'r') as f: liableSide = json.loads(f.read())
    
print('Asset and Liabilities & Equity lines have been loaded in from JSON')

Asset and Liabilities & Equity lines loaded


In [81]:
assetDict = dict([(i, np.nan) for i in assetSide])
liableDict = dict([(i, np.nan) for i in liableSide])

In [294]:
def unstructured_data(filepaths, lineDictionary, lineItems) -> tuple:
    """
    Forms unstructured data frame from .csv file(s) located in s3 bucket
    
    :param: filepaths
        filepaths from s3 that store .csv file(s) (Output/BalanceSheet/)
    :paran: lineDictionary
        dictionary of total unstructured line items and corresponding values
    :paran: lineItems
        list of line items (asset/liabilites) that will be searched for
        
    :return: tuple
        Returns a tuple, itters is a list of dataframes representing line items, error is a dictionary
        tracking all error terms that exist due to univariate dataframes 
    """
    
    # intialize list to store dataframes and errors
    itters = []
    error = {}

    # iterate through files from s3 
    for file in filepaths:
        
        # create temporary dictionary copy for storage of values
        tempDict = lineDictionary.copy()
          
        # creating two rows to track the CIK and year information released
        cik, year = file.split('/')[-1].split('-')

        tempDict['CIK'] = cik                 # CIK number for firm 
        tempDict['Year'] = '20' + year[:2]    # Year for firm filing  
        
        # retrieving downloaded files from s3 bucket
        s3.download_file(bucket, file, 'temp.pdf')
        df = pd.read_csv('temp.pdf')

        # clean dataframes for only the items and their immediate values
        cleanDF = df[df.columns[1:]] 

        # clean dataframe should be of size greater than 1
        if len(cleanDF.columns) > 1:
            
            # extract line items from each dataframe (balance sheet)
            lines = cleanDF[cleanDF.columns[0]]
            
            # filter dataframes according line items, and extract numerical values from dataframe 
            filterDF = cleanDF[np.isin(lines, lineItems)]
            filterDF = filterDF.set_index(filterDF.columns[0])             # set line items as index
            filterDF = filterDF.apply(lambda x: cleanNumeric(x.values))    # extract numerical figures

            # iterate through items from (asset or liability items)
            for item in filterDF.index:
                lineVal = filterDF.loc[item]                 # line item e.g. Cash $72,343 $71,231
                
                # check to see scope of line value, checking if multi-columns present
                # in the event we have repeating 'item' lines (e.g. 2 Prepaid expense) we sum columns  
                if type(lineVal) is not pd.Series:
                    lineVal = lineVal.sum()
                
                value = lineVal.iloc[0]                      # first column value e.g. 72343
                
                # value of line items for the adjacent column (current year)
                if ~np.isnan(value):
                    tempDict[item] = value
                    
                else:
                    try:
                        # if the first column is blank we assume the second column is filled with totals
                        value = lineVal.iloc[1]
                        
                        # if second column value is not-nan we attach those values
                        if ~np.isnan(value):
                            tempDict[item] = value
                    
                    # if no second column exists, we ignore and pass
                    except IndexError: pass
            
            # convert the dictionary values to dataframe for database construction 
            row = pd.DataFrame.from_dict(tempDict, orient='index')
            
            # append dataframe set to array transposing 
            itters.append(row.transpose())
            
        else:
            error[file] = 'Issue reading PDF'
        
        # remove local file after it has been created
        os.remove('temp.pdf')
    
    return itters, error

In [297]:
# the output directory for both asset and liability figures 
assetItters, errors = unstructured_data(obsRange, assetDict, list(assetDict.keys()))
assetDF = pd.concat(assetItters)

liableItters, errors = unstructured_data(obsRange, liableDict, list(liableDict.keys()))
liableDF = pd.concat(liableItters)

In [298]:
# re-order the CIK and Year columns to appear as the first two columns
remap = assetDF.columns[~np.isin(assetDF.columns, ['CIK', 'Year'])]
unstructADF = assetDF[np.insert(remap, [0, 0], ['CIK', 'Year'])]

In [299]:
# filter out columns with NaN values, return only values
filterNaN = unstructADF.isnull().all()
cleanCols = filterNaN[filterNaN == False].index

In [300]:
unstructADF[cleanCols].head()

,CIK,Year,"(Net of accumulated depreciation and amortization of $475,741)",Receivable from affiliate,"Furniture, equipment and leasehold improvements, at cost, less accumulated depreciation and amortization of $431 (Note 5)","Securities owned, at fair value",Investments,Deferred tax asset (Note 8),Receivable from affiliates (Note 6),Accounts receivable - trade,...,Interest and dividends receivable,Payable to Affiliate,Deposit at clearing broker,Clearing deposit with broker dealer,Receivable from related party,Total Assets,Receivable from Non-Customer,Equity securities,Accounts Receivable,"Securities, at fair value"
0,1000147,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [301]:
unstructADF[cleanCols].to_csv('unstructAssetSample.csv', index=False)

In [302]:
# re-order the CIK and Year columns to appear as the first two columns
remap = liableDF.columns[~np.isin(liableDF.columns, ['CIK', 'Year'])]
unstructLDF = liableDF[np.insert(remap, [0, 0], ['CIK', 'Year'])]

In [303]:
# filter out columns with NaN values, return only values
filterNaN = unstructLDF.isnull().all()
cleanCols = filterNaN[filterNaN == False].index

In [304]:
unstructLDF[cleanCols].head()

,CIK,Year,10 shares issued and outstanding),Accrued commissions and clearance charges,Due to To Shareholder,"Common stock, no par value, 1,000 shares authorized and 100 shares issued and outstanding",Additional paid-in-capital,Payables to affiliates (Note 6),Total liabilities and members' equity,Income taxes payable to parent,...,Bank overdrafts,Payable to Affiliate,"Securites sold, not yet purchased",MEMBERS' EQUITY,Net Income,Paid in capital,Bank overdraft,Additional Paid in Capital,Total Liabilities and Shareholder's Equity,Liability and Shareholder's Equity Liability: Due to Parent Company
0,1000147,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4200
0,1000147,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1000147,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
unstructLDF[cleanCols].to_csv('unstructLiableSample.csv', index=False)

## KMeans Clustering
**Using clustering algorithms for determing word similitaries amongst balance sheet items**

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cluster import AffinityPropagation, KMeans, AgglomerativeClustering

In [64]:
def word_matching(array: np.array, vectorizer_class, cluster_class) -> tuple:
    
    # Convert a collection of text documents to a matrix of token counts
    vectorizer = vectorizer_class
    print('Using the {} text vectorizer\n'.format(type(vectorizer_class).__name__))
    
    # produces a sparse representation of the counts using scipy.sparse.csr_matrix.
    X = vectorizer.fit_transform(array)
    
    # initialize the clustering model on words
    model = cluster_class
    
    model.fit(X)  # fit clustering model to array of strings
    
    # returns dataframe, mapping cluster labels to line items
    outDF = pd.DataFrame({'Labels': model.labels_, 'LineItems': array})
    
    print('\n{} clusters were found using {} algorithm'.format(model.labels_.max()+1, type(cluster_class).__name__))
    
    return model.labels_, outDF 

In [12]:
# reading cleaned unstructured data sets for samples 
assets = pd.read_csv('unstructAssetSample.csv')
liability = pd.read_csv('unstructLiableSample.csv')

In [13]:
assets

,CIK,Year,"(Net of accumulated depreciation and amortization of $475,741)",Receivable from affiliate,"Furniture, equipment and leasehold improvements, at cost, less accumulated depreciation and amortization of $431 (Note 5)","Securities owned, at fair value",Investments,Deferred tax asset (Note 8),Receivable from affiliates (Note 6),Accounts receivable - trade,...,Interest and dividends receivable,Payable to Affiliate,Deposit at clearing broker,Clearing deposit with broker dealer,Receivable from related party,Total Assets,Receivable from Non-Customer,Equity securities,Accounts Receivable,"Securities, at fair value"
0,1000147,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000147,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000147,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000147,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000147,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,1002201,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,1002201,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,1002201,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,1002201,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**The following clustering model specifications are outlined below:**

- **array:** We select the columns from either asset or liability tables, not including CIK or Year components.
- **vectorizer_class:** We implement a standard TfidVectorizer class, converting the collection of strings to a matrix of TF-IDF features.

- **cluster_class:** We fit a KMeans clustering algorithm, employing 30 cluster using the the classic expectation–maximization algorithm (full), we control the random state and provided verbosity to model fits

In [225]:
clusters = 30
label, df = word_matching(array = assets.columns[2:], 
                          vectorizer_class = TfidfVectorizer(lowercase=True), 
                          cluster_class = KMeans(n_clusters=clusters, verbose=1, random_state=1, algorithm='full'))

Using the TfidfVectorizer text vectorizer

Initialization complete
Iteration  0, inertia 539.449
Iteration  1, inertia 332.601
Iteration  2, inertia 326.984
Iteration  3, inertia 325.168
Iteration  4, inertia 324.421
Iteration  5, inertia 323.689
Iteration  6, inertia 323.606
Iteration  7, inertia 323.538
Converged at iteration 7: center shift 0.000000e+00 within tolerance 2.142878e-07
Initialization complete
Iteration  0, inertia 523.115
Iteration  1, inertia 335.185
Iteration  2, inertia 328.181
Iteration  3, inertia 326.516
Iteration  4, inertia 325.754
Iteration  5, inertia 325.611
Converged at iteration 5: center shift 0.000000e+00 within tolerance 2.142878e-07
Initialization complete
Iteration  0, inertia 528.214
Iteration  1, inertia 332.397
Iteration  2, inertia 325.364
Iteration  3, inertia 321.758
Iteration  4, inertia 321.069
Converged at iteration 4: center shift 0.000000e+00 within tolerance 2.142878e-07
Initialization complete
Iteration  0, inertia 528.556
Iteration  1, i

In [214]:
df

,Labels,LineItems
0,24,(Net of accumulated depreciation and amortizat...
1,27,Receivable from affiliate
2,13,"Furniture, equipment and leasehold improvement..."
3,12,"Securities owned, at fair value"
4,20,Investments
...,...,...
538,28,Total Assets
539,22,Receivable from Non-Customer
540,25,Equity securities
541,3,Accounts Receivable


## Latent Dirichlet Allocation with online variational Bayes algorithm
**A Topic Model for root-word detection from a list of similar strings** 

Our goal here is to determine whether using LDA decomposition is useful in determining label names for clustering classes

In [216]:
from sklearn.decomposition import LatentDirichletAllocation

In [217]:
# sample to determine adequate label name from cluster array names
sample = df[df.Labels == 0]['LineItems'].values

In [218]:
def label_LDA(names, LDA_model):
    assert type(LDA_model).__name__ is 'LatentDirichletAllocation', 'Model restriction, only imposed LDA'
    
    print('Clustered names provided are:\n\n', names, '\n')
    labelNames = []
    
    # vectorize the text (string) data to be used by Latent Dirichlet Allocation algorithm
    text_mdl = TfidfVectorizer()
    vec_text = text_mdl.fit_transform(names)
    
    # feature names extracted from the TfidVectorizer
    feature_names = text_mdl.get_feature_names()
  
    # fit Laten Dirichlet Allocation model 
    mdl = LDA_model
    mdl.fit(vec_text)
    
    print()
    # print out label names determined from LDA
    for topic in mdl.components_:
        message = "Topic determined: "
        message += " ".join([feature_names[i]
                             # select the 5 most important feature names from largest to smallest order
                             for i in topic.argsort()[:-5 - 1:-1]])
        print(color.BOLD, message, color.END)
        labelNames.append(message)
    print()
    
    return labelNames

In [224]:
sample_label = label_LDA(sample, LatentDirichletAllocation(n_components=1, verbose=1, random_state=1))

Clustered names provided are:

 ['Deposits at clearing brokers' 'Clearing firm deposits' 'NASD warrants'
 'Deduct: Ownership equity not allowable for net capital'
 'Right of use asset - operating lease' 'Right-of-use asset' 'Prepaids'
 'Foreign currencies' 'Computer Equipment' 'Clearing charges payable'
 'Securities purchased under agreement to resell'
 'Money market funds held at clearing broker' 'Loan' 'December 31,'
 'Fails to deliver' 'Fail to Deliver' 'Deposits at clearing organization'
 'Finantia USA Ltd. Statement of Financial Condition December 31, 2002'
 'Exchange memberships' 'Concessions and fees receivable'
 'Advanced compensation arrangements, net (Note 4)'
 'Good Faith Deposit Account'
 'Securities purchased under agreements to resell' 'Investment'
 '$15,817 in 2001 and $9,217 in 2000' 'Vehicles' 'Vehicle' 'Goodwill'
 'Investment - NASDAQ' 'Dividends Payable' 'Time deposits'
 'Loans and exchange' 'Deposits at clearing firms'
 'Deposits with clearing organizations' 'Securi

Continuing this current assesment of topic detection, as illustrated above, we extend this to the entire subset of items computed from the clustering algorthim employed to group asset/liability line items. (Refer to KMeans Model)

In [220]:
for i in range(df.Labels.max()+1):
    print('Label', i)
    cluster_cells = df[df.Labels == i]['LineItems'].values
    print()
    _ = label_LDA(cluster_cells, LatentDirichletAllocation(n_components=1, verbose=1, random_state=1))
    print()

Label 0

Clustered names provided are:

 ['Deposits at clearing brokers' 'Clearing firm deposits' 'NASD warrants'
 'Deduct: Ownership equity not allowable for net capital'
 'Right of use asset - operating lease' 'Right-of-use asset' 'Prepaids'
 'Foreign currencies' 'Computer Equipment' 'Clearing charges payable'
 'Securities purchased under agreement to resell'
 'Money market funds held at clearing broker' 'Loan' 'December 31,'
 'Fails to deliver' 'Fail to Deliver' 'Deposits at clearing organization'
 'Finantia USA Ltd. Statement of Financial Condition December 31, 2002'
 'Exchange memberships' 'Concessions and fees receivable'
 'Advanced compensation arrangements, net (Note 4)'
 'Good Faith Deposit Account'
 'Securities purchased under agreements to resell' 'Investment'
 '$15,817 in 2001 and $9,217 in 2000' 'Vehicles' 'Vehicle' 'Goodwill'
 'Investment - NASDAQ' 'Dividends Payable' 'Time deposits'
 'Loans and exchange' 'Deposits at clearing firms'
 'Deposits with clearing organizations


 Topic determined: from receivable clearing dealers broker 


Label 8

Clustered names provided are:

 ['Clearing broker' 'Deposit with clearing broker-dealer'
 'Due from broker-dealer (Notes 1 and 11)'
 'U.S. Treasury bill on deposit with broker-dealer, at market value'
 'Deposits with clearing broker-dealer'
 'Receivable from clearing broker dealer'
 'Receivables from clearing broker' 'Receivable from broker'
 'Receivable from broker-dealer' 'Due from broker-dealer (Notes 1 and 10)'
 'Broker-dealer commissions receivable'
 'Receivable from clearing broker/dealer' 'Deposits at clearing broker'
 'RECEIVABLE FROM BROKER/DEALER' 'Receivable from broker/dealer'
 'Broker-dealer commission receivable' 'Investment in broker-dealer'
 'Receivable from clearing broker'
 'Deposits with and receivables from clearing broker'
 'Broker-dealer commissions' 'Payable to clearing broker'
 'BROKER OR DEALER: BLACKHAWK FINANCIAL L.LC.'
 'Receivable from clearing broker-dealer'
 'Clearing deposit with bro

iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10

 Topic determined: receivables customers other from brokerage 


Label 16

Clustered names provided are:

 ['FURNITURE AND FIXTURES' 'Equipment and furniture, net'
 'Office furniture and equipment' 'Furniture & office equipment, net'
 'Office furniture and equipment, net of accumulated depreciation of $13,006'
 'Furniture and office equipment, net'
 'Office furniture and equipment, net of accumulated depreciation of $12,194'
 'NET FURNITURE AND FIXTURES'
 'Office furniture and equipment, net of accumulated depreciation of $7,947'
 'Furniture and equipment' 'NET BUILDING, FURNITURE AND FIXTURES'
 'Office furniture and equipment, net of accumulated depreciation of $9,542'
 'Furniture, fixtures and equipment' 'Net Equipment and Furniture'
 'Equipment, net'
 'Office furniture and equipment, net of accumulated depreci

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10

 Topic determined: of depreciation amortization and accumulated 


Label 25

Clustered names provided are:

 ['Trading Securities, Not Readily Marketable, at Fair Value'
 'Trading securities' 'Deposits paid for securities borrowed'
 'Not readily marketable securities' 'Marketable debt securities'
 'Marketable Securities @ FMV' 'Securities borrowed'
 'Marketable Securities Prepaid Rent'
 'Marketable securities (Notes 2 and 11)'
 'Securities owned, not readily marketable'
 'Marketable equity securities, available for sale'
 'Marketable Securities' 'Marketable securities' 'Equity securities'] 

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_it

## Classification Model
**Construct a classification model to match user defined categories to established balance sheet items**

In [230]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [228]:
for i in range(clusters):
    print('Label', i)
    print(df[df.Labels == i].LineItems.values)
    print()

Label 0
['Deposits at clearing brokers' 'Clearing firm deposits' 'NASD warrants'
 'Deduct: Ownership equity not allowable for net capital'
 'Right of use asset - operating lease' 'Right-of-use asset' 'Prepaids'
 'Foreign currencies' 'Computer Equipment' 'Clearing charges payable'
 'Securities purchased under agreement to resell'
 'Money market funds held at clearing broker' 'Loan' 'December 31,'
 'Fails to deliver' 'Fail to Deliver' 'Deposits at clearing organization'
 'Finantia USA Ltd. Statement of Financial Condition December 31, 2002'
 'Exchange memberships' 'Concessions and fees receivable'
 'Advanced compensation arrangements, net (Note 4)'
 'Good Faith Deposit Account'
 'Securities purchased under agreements to resell' 'Investment'
 '$15,817 in 2001 and $9,217 in 2000' 'Vehicles' 'Vehicle' 'Goodwill'
 'Investment - NASDAQ' 'Dividends Payable' 'Time deposits'
 'Loans and exchange' 'Deposits at clearing firms'
 'Deposits with clearing organizations' 'Security deposits'
 'clearing 

In [ ]:
# split dependent and independent variables into training and testing windows
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [231]:
# develop Pipeline to run against basket of models
pipe = Pipeline([('naives', MultinomialNB()), ('svc', LinearSVC())])

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)' (type <class 'sklearn.naive_bayes.MultinomialNB'>) doesn't

## Consolidating Clusters to Structured Database
**Structured convergece for Unstructured data labels**

In [133]:
def structured_data(unstructured_df:pd.DataFrame, cluster_df:pd.DataFrame, label_names:np.array) -> pd.DataFrame:
    """
    Constructs a structured dataset from an unstructured column set
    
    :param: unstructured_df (type pandas.DataFrame)
        unstuructured pandas dataframe with loose column construction 
    :param: cluster_df (type pandas.DataFrame)
        a pandas dataframe of clustered labels and corresponding line items
    :param: (type numpy array)
        all corresponding cluster labels cirresponding with 'cluster_df' parameter
        
    :return: (type pandas DataFrame)
    """
    
    structured_df = pd.DataFrame()
    
    # assume that the there exists columns 'CIK' and 'Year' for unstructured data
    structured_df = unstructured_df[['CIK', 'Year']]
    
    for label in label_names:
        data = cluster_df[cluster_df['Labels'] == label]['LineItems']     # filter by corresponding cluster
        
        # sum all columns, across row and map to structured dataframe
        structured_df[label] = unstructured_df[data.values].sum(axis=1)
        
    return structured_df

In [134]:
structured_data(assets, df, df['Labels'].unique())

,CIK,Year,3,2,4,6,11,14,10,0,9,13,7,5,1,12,8
0,1000147,2002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74600.0,0.0,0.0
1,1000147,2003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75800.0,0.0,0.0
2,1000147,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76500.0,0.0,0.0
3,1000147,2005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000147,2006,36301.0,0.0,332085.0,0.0,0.0,0.0,15788.0,0.0,2005.0,0.0,0.0,42132.0,253652.0,0.0,395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1000320,2009,0.0,0.0,0.0,0.0,13000.0,0.0,0.0,653311.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,1000320,2010,0.0,0.0,0.0,0.0,15000.0,0.0,0.0,88052.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,1000320,2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,1000320,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
